In [1]:
import pandas as pd
import requests
import time

### Create Nodes for Mapping

In [3]:
raw = pd.read_csv('./data/mergedData/reducedStops.csv')
df = raw.drop(['Unnamed: 0', 'id'], axis=1)


In [5]:

def f(x):
     return pd.Series(dict(lat = x['lat'].mean(),
                           long = x['long'].mean(),
                           transtype = "[%s]" % ', '.join(x['type'].unique())))

df = df.groupby('assigned').apply(f)

In [25]:
def concatenatorator(x):
    x = x[1:-1]
    x = x.split(',')
    x = [y.strip() for y in x]
    x.sort()
    tmp = str()
    for i in x:
        if i == "bikeStop":
            tmp = tmp+'bi'
        elif i == "busStop":
            tmp = tmp+'bu'
        elif i == "trainStop":
            tmp = tmp+'tr'
    return tmp

finaldf = df.copy()
finaldf['assigned'] = df.index.values
finaldf['convertTrans'] = finaldf['transtype'].apply(concatenatorator)
finaldf['mikeStationId'] = finaldf['convertTrans']+finaldf['assigned'].astype(str)
finaldf.drop(['transtype', 'convertTrans'], axis=1, inplace=True)
#finaldf.to_csv("./data/mergedData/nodes.csv", index=False)

In [27]:
finaldf2 = finaldf.reset_index()

finaldf.head()

,lat,long,assigned,mikeStationId
assigned,,,,
0,40.889377,-73.898421,0,butr0
1,40.884734,-73.900801,1,butr1
2,40.878911,-73.904831,2,butr2
3,40.874548,-73.909565,3,butr3
4,40.869427,-73.915577,4,butr4


### Create Edges for Mapping

In [8]:
def concatenatorator2(x):
    tmp = str()
    if x == "bikeStop":
        tmp = 'bi'
    elif x == "busStop":
        tmp = 'bu'
    elif x == "trainStop":
        tmp = 'tr'
    return tmp

# use the raw loaded data to create a complete list of all
# mta station ids and mike's assigned ids
# this is different then the groupby approach above.
mergetable = raw.drop(['Unnamed: 0', 'lat', 'long'], axis=1)
mergetable['type2'] = mergetable['type'].apply(concatenatorator2)
mergetable['type'] = mergetable['type2']
mergetable['mtaTypeStatId'] = mergetable['type']+mergetable['id'].astype(str)
mergetable.drop('type2', axis=1, inplace=True)
mergetable.head()

,type,assigned,id,mtaTypeStatId
0,tr,0,101,tr101
1,tr,0,101N,tr101N
2,tr,0,101S,tr101S
3,tr,1,103,tr103
4,tr,1,103N,tr103N


In [9]:
trips = pd.read_csv('./data/mergedData/busTrainTrips.csv')


/home/michael/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2902: DtypeWarning: Columns (1,2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [10]:
joined_dfs = mergetable.merge(finaldf, how='inner', on='assigned')
joined_dfs.head()

,type,assigned,id,mtaTypeStatId,lat,long,mikeStationId
0,tr,0,101,tr101,40.889377,-73.898421,butr0
1,tr,0,101N,tr101N,40.889377,-73.898421,butr0
2,tr,0,101S,tr101S,40.889377,-73.898421,butr0
3,bu,0,100587,bu100587,40.889377,-73.898421,butr0
4,tr,1,103,tr103,40.884734,-73.900801,butr1


In [11]:
#merge old and new id's in trips

joined_dfs['id'] =joined_dfs['id'].apply(lambda x: str(x).split('.')[0])
trips['mta_start_id'] = trips['mta_start_id'].apply(lambda x: str(x).split('.')[0])
trips['mta_stop_id'] = trips['mta_stop_id'].apply(lambda x: str(x).split('.')[0])


keyValuePair= joined_dfs[['id','mikeStationId']]
keyValuePair = keyValuePair.drop_duplicates()
ids = list(keyValuePair['id'].values)
reducedId = list(keyValuePair['mikeStationId'].values)

switchingDic = dict(zip(ids,reducedId))

trips['source']=trips['mta_start_id'].apply(lambda x: switchingDic[str(x)])
trips['target']=trips['mta_stop_id'].apply(lambda x: switchingDic[str(x)])


In [12]:
tripsReduced = trips[['source','target','duration']]
tripsReduced.to_csv('./data/mergedData/reducedTrips.csv')

In [21]:
#reduce trips 
tripsReduced2 = tripsReduced.groupby(['source','target']).mean()
tripsReduced2= tripsReduced2.reset_index()
tripsReduced2.to_csv('./data/mergedData/averagedTripDuration.csv')

In [120]:
joined_deduped = joined_dfs.drop_duplicates()
print joined_deduped.head()
# Create a Start Station Version
startStation = joined_deduped[['mtaTypeStatId','lat','long','mikeStationId']].copy()
startStation.columns = ['start_mtaTypeStatId','start_lat','start_long','start_mikeStationId']
# Create an End Station Version
endStation = joined_deduped[['mtaTypeStatId','lat','long','mikeStationId']].copy()
endStation.columns = ['end_mtaTypeStatId','end_lat','end_long','end_mikeStationId']
endStation.head()

  type  assigned      id mtaTypeStatId        lat       long mikeStationId
0   tr         0     101         tr101  40.889377 -73.898421         butr0
1   tr         0    101N        tr101N  40.889377 -73.898421         butr0
2   tr         0    101S        tr101S  40.889377 -73.898421         butr0
3   bu         0  100587      bu100587  40.889377 -73.898421         butr0
4   tr         1     103         tr103  40.884734 -73.900801         butr1


,end_mtaTypeStatId,end_lat,end_long,end_mikeStationId
0,tr101,40.889377,-73.898421,butr0
1,tr101N,40.889377,-73.898421,butr0
2,tr101S,40.889377,-73.898421,butr0
3,bu100587,40.889377,-73.898421,butr0
4,tr103,40.884734,-73.900801,butr1


In [129]:
# merge together the old and new ids
final = pd.merge(trips, startStation, left_on="mta_start_id", right_on='start_mtaTypeStatId', how='left')
final2 = final.merge(endStation, how='left', left_on='mta_stop_id', right_on='end_mtaTypeStatId')
final2.shape
final2.head(1)

,Unnamed: 0,mta_stop_id,mta_start_id,type,duration,start_mtaTypeStatId,start_lat,start_long,start_mikeStationId,end_mtaTypeStatId,end_lat,end_long,end_mikeStationId
0,0,902N,901N,trainStop,1.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [121]:
#joined_dfs_dropped = joined_dfs.drop(['id', 'type', 'assigned', 'mtaTypeStatId'], axis=1)
joined_dfs_grouped = joined_dfs_dropped.groupby('mikeStationId').mean()
joined_dfs_grouped['id'] = joined_dfs_grouped.index
joined_dfs_grouped = joined_dfs_grouped.reset_index()
joined_dfs_grouped.drop(['mikeStationId'],axis =1, inplace =True)
joined_dfs_grouped.to_csv('./data/mergedData/nodes.csv')
joined_dfs_grouped.head()


,lat,long,id
0,40.657916,-73.925333,bu1000
1,40.659266,-73.927177,bu1001
2,40.660910,-73.929009,bu1002
3,40.663266,-73.931711,bu1003
4,40.664516,-73.937023,bu1004


# NOTES
At this point we ran a map-reduce algorithm , busReduceTripsmap.py and busReduce.py to first map old station id's to reducedStationId's and then compute average trip duration between every pair of reducedStationID's


In [79]:
import json
import json
from shapely.geometry import shape, Point
with open('/Users/matthewdunn/Dropbox/NYU/Spring2016/BigData/GroupProject/nyccensustracts.json', 'r') as f:
    js = json.load(f)


def geocoder(lat, lon):
    point = Point(lon, lat)

    # check each polygon to see if it contains the point
    for feature in js['features']:
        geometry = feature.get('geometry')
        polygon = shape(geometry)
        if polygon.contains(point):
                properties = feature.get('properties')
                census_tract = properties.get('BoroCT2010')
                return census_tract

IOError: [Errno 2] No such file or directory: '/Users/matthewdunn/Dropbox/NYU/Spring2016/BigData/GroupProject/nyccensustracts.json'

In [ ]:
censustracts = []
for i, row in joined_dfs_grouped.iterrows():
    censustract = geocoder(row.lat, row['long'])
    censustracts.append(censustract)
censustracts

In [ ]:
allpairsBus = pd.read_csv("./data/allPairDistancesBusTrain.csv")
allpairsBus.head()

In [ ]:
allpairsBus_drop = allpairsBus.drop(['assigned', 'Unnamed: 0.1'], 1)
allpairsBus_drop = allpairsBus_drop.drop([0, 1])
cols = list(allpairsBus_drop.columns.values)
cols = cols[1:]
duration_df = pd.melt(allpairsBus_drop, id_vars=['Unnamed: 0'], value_vars=cols)
duration_df.columns = ['mikestartId', 'mikeendId', 'duration']
duration_df = duration_df[duration_df.mikestartId != duration_df.mikeendId]
duration_df.head()

In [ ]:
joined_dfs_grouped['censustract'] = censustracts
joined_dfs_grouped.head()

In [68]:
merged_duration = duration_df.merge(joined_dfs_grouped, how='left', left_on='mikestartId', right_index=True)
merged_duration.columns = ['mikestartId','mikeendId','duration','start_lat','start_long','start_censustract']
merged_duration.head()

,mikestartId,mikeendId,duration,start_lat,start_long,start_censustract
1,bibu101,bibu100,12,40.696078,-73.967684,054300
2,bibu106,bibu100,9,40.753307,-73.978567,009400
3,bibu108,bibu100,8,40.717117,-74.006062,003300
4,bibu109,bibu100,10,40.696508,-73.991280,000502
5,bibu11,bibu100,8,40.698154,-73.980527,021100


In [69]:
merged_duration_final = merged_duration.merge(joined_dfs_grouped, how='left', left_on='mikeendId', right_index=True)

In [72]:
merged_duration_final.columns = ['mikestartId','mikeendId','duration','start_lat','start_long','start_censustract','end_lat','end_long','end_censustract']
merged_duration_final.head()

,mikestartId,mikeendId,duration,start_lat,start_long,start_censustract,end_lat,end_long,end_censustract
1,bibu101,bibu100,12,40.696078,-73.967684,054300,40.721974,-73.988963,003001
2,bibu106,bibu100,9,40.753307,-73.978567,009400,40.721974,-73.988963,003001
3,bibu108,bibu100,8,40.717117,-74.006062,003300,40.721974,-73.988963,003001
4,bibu109,bibu100,10,40.696508,-73.991280,000502,40.721974,-73.988963,003001
5,bibu11,bibu100,8,40.698154,-73.980527,021100,40.721974,-73.988963,003001


In [65]:
from haversine import haversine
def manhattandist(start_lat, start_lon, end_lat, end_lon):
    start_station_location = (start_lat, start_lon)
    end_station_location = (end_lat, end_lon)
    return haversine(start_station_location, end_station_location, miles=True)

In [84]:
distances = []
for i, row in merged_duration_final.iterrows():
    distance = manhattandist(row.start_lat, row.start_long, row.end_lat, row.end_long)
    distances.append(distance)
len(distances)

56874222

In [85]:
merged_duration_final['distance'] = distances
merged_duration_final.head()

,mikestartId,mikeendId,duration,start_lat,start_long,start_censustract,end_lat,end_long,end_censustract,distance
1,bibu101,bibu100,12,40.696078,-73.967684,054300,40.721974,-73.988963,003001,2.107982
2,bibu106,bibu100,9,40.753307,-73.978567,009400,40.721974,-73.988963,003001,2.232244
3,bibu108,bibu100,8,40.717117,-74.006062,003300,40.721974,-73.988963,003001,0.956258
4,bibu109,bibu100,10,40.696508,-73.991280,000502,40.721974,-73.988963,003001,1.763686
5,bibu11,bibu100,8,40.698154,-73.980527,021100,40.721974,-73.988963,003001,1.704060


In [ ]:
merged_duration_final.shape

In [ ]:
merged_duration_final.to_csv('/Users/matthewdunn/Dropbox/NYU/Spring2016/BigData/GroupProject/data/mikeCTsAvgLatLonDist.csv')

## Now Create the Bus Sheet for Map Reduce

In [ ]:
merged_raw = pd.read_csv('/Users/matthewdunn/Dropbox/NYU/Spring2016/BigData/GroupProject/data/mta_bus/mergedRoutesStops.csv')

In [ ]:
merged_raw.shape

In [ ]:
trips_dropped = merged_raw.drop(['Unnamed: 0', 'trip_id', 'start_lat', 'start_lon', 'end_lon', 'end_lat', 'arrival_time', 'departure_time'], axis=1)
trips_dropped['end_location_id'] = trips_dropped['end_location_id'].astype(int)
trips_dropped['start_mtaTypeStatId'] = 'bu'+trips_dropped['start_location_id'].astype(str)
trips_dropped['end_mtaTypeStatId'] = 'bu'+trips_dropped['end_location_id'].astype(str)
trips_dropped.drop(['end_location_id', 'start_location_id'], axis=1, inplace=True)
print trips_dropped.head()
trips_dropped.shape

In [ ]:
# Dedupe the Merge Key between MTA and Mikes ID
joined_deduped = joined_dfs.drop_duplicates()
print joined_deduped.head()
# Create a Start Station Version
startStation = joined_deduped[['mtaTypeStatId','lat','long','mikeStationId']].copy()
startStation.columns = ['start_mtaTypeStatId','start_lat','start_long','start_mikeStationId']
# Create an End Station Version
endStation = joined_deduped[['mtaTypeStatId','lat','long','mikeStationId']].copy()
endStation.columns = ['end_mtaTypeStatId','end_lat','end_long','end_mikeStationId']
print endStation.head()

In [ ]:
# validating shit works
# joined_deduped.loc[joined_deduped.id == '901N']
# print any(joined_deduped.mtaTypeStatId == 'bu901N')
# print any(startStation.start_mtaTypeStatId == '901N')

In [ ]:
final = pd.merge(trips_dropped, startStation, on='start_mtaTypeStatId', how='left')
final2 = final.merge(endStation, how='left', left_on='end_mtaTypeStatId', right_on='end_mtaTypeStatId')
final2.shape

In [ ]:
fianl2_dropped = final2.drop(['start_mtaTypeStatId', 'end_mtaTypeStatId', 'start_lat', 'start_long','end_lat', 'end_long'], axis=1)
print fianl2_dropped.head()
print fianl2_dropped.shape

In [ ]:
fianl2_dropped['key'] = fianl2_dropped['start_mikeStationId']+'_'+fianl2_dropped['end_mikeStationId']
mapreduce = fianl2_dropped.drop(['start_mikeStationId', 'end_mikeStationId'], axis=1)
mapreduce = mapreduce[['key', 'duration']]
mapreduce['duration'] = mapreduce['duration']*60
print mapreduce.head()
print mapreduce.shape
mapreduce.to_csv('/Users/matthewdunn/Dropbox/NYU/Spring2016/BigData/GroupProject/data/mta_bus/mapreduce.csv', index=False, header=False)

### Used to write out small sample files

In [ ]:
shorter = pd.read_csv('/Users/matthewdunn/Dropbox/NYU/Spring2016/BigData/GroupProject/data/allPairDistancesBusTrain.csv', nrows=100)

In [ ]:
shorter.to_csv('/Users/matthewdunn/Dropbox/NYU/Spring2016/BigData/GroupProject/data/allPairDistancesBusTrain_100.csv', index=False)

## Now Create the Train Sheet for Map Reduce

In [ ]:
merged_raw_train = pd.read_csv('/Users/matthewdunn/Dropbox/NYU/Spring2016/BigData/GroupProject/data/mta_train/mergedTrainRoutesStops.csv')

In [ ]:
merged_raw_train.shape

In [ ]:
trips_dropped_train = merged_raw_train.drop(['Unnamed: 0', 'trip_id', 'start_lat', 'start_lon', 'end_lon', 'end_lat', 'arrival_time', 'departure_time'], axis=1)
# trips_dropped_train['end_location_id'] = trips_dropped_train['end_location_id'].astype(int)
trips_dropped_train['start_mtaTypeStatId'] = 'tr'+trips_dropped_train['start_location_id'].astype(str)
trips_dropped_train['end_mtaTypeStatId'] = 'tr'+trips_dropped_train['end_location_id'].astype(str)
trips_dropped_train.drop(['end_location_id', 'start_location_id'], axis=1, inplace=True)
print trips_dropped_train.head()
print trips_dropped_train.shape

In [ ]:
any(joined_dfs.mtaTypeStatId == 'tr901N')

In [ ]:
final_train = pd.merge(trips_dropped_train, startStation, on='start_mtaTypeStatId', how='left')
final2_train = final_train.merge(endStation, how='left', left_on='end_mtaTypeStatId', right_on='end_mtaTypeStatId')
final2_train.shape

In [ ]:
fianl2_train_dropped = final2_train.drop(['start_mtaTypeStatId', 'end_mtaTypeStatId', 'start_lat', 'start_long','end_lat', 'end_long'], axis=1)
print fianl2_train_dropped.head()
print fianl2_train_dropped.shape

In [ ]:
fianl2_train_dropped['key'] = fianl2_train_dropped['start_mikeStationId']+'_'+fianl2_train_dropped['end_mikeStationId']
mapreduce_train = fianl2_train_dropped.drop(['start_mikeStationId', 'end_mikeStationId'], axis=1)
mapreduce_train = mapreduce_train[['key', 'duration']]
mapreduce_train['duration'] = mapreduce_train['duration']*60
print mapreduce_train.head()
print mapreduce_train.shape
mapreduce_train.to_csv('/Users/matthewdunn/Dropbox/NYU/Spring2016/BigData/GroupProject/data/mta_train/mapreduce.csv', index=False, header=False)

In [ ]:
bus_train_map_reduce = pd.concat([mapreduce_train, mapreduce], ignore_index=True)

In [ ]:
print bus_train_map_reduce.head()
print bus_train_map_reduce.shape
bus_train_map_reduce.to_csv('/Users/matthewdunn/Dropbox/NYU/Spring2016/BigData/GroupProject/data/bus_train_mapreduce.csv', index=False, header=False)